In [2]:
import pathlib
import os
import datetime
import time

import numpy as np
# import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, Model, utils, optimizers, losses
import tensorflow as tf
import matplotlib.pyplot as plt


c:\Users\natha\anaconda3\envs\ml\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# model expected shape=(None, 224, 224)
IMG_SIZE = (224, 224)
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
flowers_dir = 'small_flower_dataset/'
EPOCHS = 20

In [4]:

batch_size = 32
train_ds = tf.keras.utils.image_dataset_from_directory(
                flowers_dir,
                labels='inferred',
                label_mode='int',
                class_names=None,
                color_mode='rgb',
                batch_size=batch_size,
                image_size=IMG_SIZE,
                shuffle=True,
                seed=2,
                validation_split=0.3,
                subset="training",
                interpolation='bilinear',
                follow_links=False,
                crop_to_aspect_ratio=False)
val_ds = tf.keras.utils.image_dataset_from_directory(
                flowers_dir,
                labels='inferred',
                label_mode='int',
                class_names=None,
                color_mode='rgb',
                batch_size=batch_size,
                image_size=IMG_SIZE,
                shuffle=True,
                seed=2,
                validation_split=0.3,
                subset="validation",
                interpolation='bilinear',
                follow_links=False,
                crop_to_aspect_ratio=False)
class_names = train_ds.class_names

testing_ds = val_ds.take(2)
val_ds = val_ds.skip(2)

    

# Configure dataset for performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
testing_ds = testing_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Standardize the data
# The RGB channel values are in the [0, 255] range. 
# This is not ideal for a neural network; in general you should seek to make your input values small.
normalization_layer = layers.Rescaling(1./255)
normalized_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_testing_ds = testing_ds.map(lambda x, y: (normalization_layer(x), y))

train_ds = normalized_train_ds
val_ds = normalized_val_ds
testing_ds = normalized_testing_ds

Found 886 files belonging to 5 classes.
Using 621 files for training.
Found 886 files belonging to 5 classes.
Using 265 files for validation.


In [5]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=True, weights="imagenet")
    
# Freeze layer exclude new layer
for layer in base_model.layers:
    layer.trainable=False



In [6]:
feature_extractor = Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)
# feature_extractor.summary()

#feature extraction
train_ds_features = feature_extractor.predict(train_ds)
val_ds_features = feature_extractor.predict(val_ds)
test_ds_features = feature_extractor.predict(testing_ds)

2/2 [==============================] - 1s 359ms/step


In [7]:
train_ds_features.shape
val_ds_features.shape
test_ds_features.shape

(64, 1280)

In [8]:
# extract the labels from the _ds batches and concatenate them into one array
train_labels = np.concatenate([y for x, y in train_ds], axis=0)
val_labels = np.concatenate([y for x, y in val_ds], axis=0)
test_labels = np.concatenate([y for x, y in testing_ds], axis=0)

In [9]:
inputs = tf.keras.Input(shape=(1280), name="img")
# x = layers.GlobalAveragePooling2D(inputs)
x = layers.Dense(256, activation='relu')(inputs)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(5, activation='softmax')(x)

model = Model(inputs = inputs, outputs = outputs)

model.summary()

model.compile(
    optimizer=optimizers.SGD(learning_rate=0.1, momentum=0, nesterov=False),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"])
    
history = model.fit(train_ds_features, train_labels, epochs=EPOCHS, validation_data=(val_ds_features, val_labels))



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 img (InputLayer)            [(None, 1280)]            0         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 329,221
Trainable params: 329,221
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
20/20 [==============================] - 1s 19ms/step - loss: 1.8755 - accuracy: 0.2158 - val_loss: 1.5065 - val_accuracy: 0.3632
Epoch 2/20
20/20 [==========================

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']


epochs_range = range(EPOCHS)
# plt.figure(figsize=(8, 8))



plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')



plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()